Harris Economic Order Quantity (EOQ) 

`y` is the optimal order quantity

`m` is the units per month

`s` is the setup cost of an order

`c` is the unit price of items in the stock

In [1]:
from dataclasses import dataclass
import jax.numpy as jnp


def ooq(m: float, c: float, s: float) -> int:
    """
    Optimal Order Quantity.
    
    Economic order quantity function defined by Harris, 1990. 
    
    The number 240 is an "interest charge" per unit, per month, 
    corresponding to a yearly interest rate of 10 percent.
    
    :param x: dataclass representing a vector of model inputs. 
    :return: optimal order quantity
    """
    return jnp.sqrt(240 * m * s / c)

X = {'m': 100, 'c': 5, 's': 1}
y = ooq(X['m'], X['c'], X['s'])
print(y)

69.282036


## Sensitivity Analysis with Finite Differences
### One-at-a-Time (OAT) Methods

We assign a base case `x0` and a sensitivity case `x_plus` to the model 
inputs. 

We change each model input, one at a time, by 10%. Then, we compare the 
resulting delta_y outputs (one for each changed parameter). 

In [2]:
 # Initial parameters                                                                                        
 x0 = {'m': 1230.0, 'c': 0.0135, 's': 2.15}                                                                    
 y0 = ooq(x0['m'], x0['c'], x0['s'])                                                      
                                                                                                             
 # Parameters with changes                                                                                   
 x_plus_m = {'m': 1353, 'c': 0.0135, 's': 2.15}                                                              
 x_plus_s = {'m': 1230, 'c': 0.01485, 's': 2.15}                                                             
 x_plus_c = {'m': 1230, 'c': 0.0135, 's': 2.365}                                                             
 x_plus = {'m': 1353, 'c': 0.01485, 's': 2.365}                                                              
                                                                                                             
 # Calculate changes in output                                                                               
 delta_y_m = ooq(x_plus_m['m'], x_plus_m['c'], x_plus_m['s']) - y0                        
 delta_y_s = ooq(x_plus_s['m'], x_plus_s['c'], x_plus_s['s']) - y0                        
 delta_y_c = ooq(x_plus_c['m'], x_plus_c['c'], x_plus_c['s']) - y0                        
                                                                                                             
 # Store deltas in a list                                                                                    
 deltas = [delta_y_m, delta_y_s, delta_y_c]                                                                  
                                                                                                             
 # Print results                                                                                             
 print("Base case:", y0)                                                                                     
 print("Changes in model output with respect to m, s, and c:\n", deltas)                                     
                                                                                

Base case: 6856.627
Changes in model output with respect to m, s, and c:
 [Array(334.66406, dtype=float32, weak_type=True), Array(-319.08984, dtype=float32, weak_type=True), Array(334.66406, dtype=float32, weak_type=True)]


This indicates that when `m` is increased by 10%, the model output increases
 by 335 units. When `s` is increased, the model output decreases by 319 
 units, and so on. 

In [3]:
delta_y_plus = ooq(x_plus['m'], x_plus['c'], x_plus['s']) - y0
sum_of_delta = jnp.sum(jnp.array(deltas))

print("Change in y from x0 to x_plus:", delta_y_plus)
print("Sum of individual changes:", sum_of_delta)

Change in y from x0 to x_plus: 334.66406
Sum of individual changes: 350.23828


**Note that these are not equal**. This is due to the fact that this method of 
sensitivity analysis does not take interaction effects into account. 

## Scenario Decomposition
We start with a base case `x0`, best case `x_plus`, and worst case `x_minus`. 

## Differentiation-based Methods


In [4]:
import jax 
import jax.numpy as jnp

df_dm = jax.grad(ooq, argnums=0)(x0['m'], x0['c'], x0['s'])
print(df_dm)

2.7872467


In [6]:
df_dc = jax.grad(ooq, argnums=1)(x0['m'], x0['c'], x0['s'])
print(df_dc)

-253949.12


In [7]:
df_ds = jax.grad(ooq, argnums=2)(x0['m'], x0['c'], x0['s'])
print(df_ds)

1594.5643


Partial derivatives are not comparable, because they are denominated in different units. 

Thus, we define the Differential Importance Measure, D_i, which indicates the DIM of the variable x_i in our input vector. 

The elasticity of x_i, denoted E_i, is an intermediate calculation. 

D_i = E_i / \sum_{j=1}^n E_j, or D_i = E_i / (sum of all elasticities). 

In [15]:
from typing import Callable

def elasticity(x: dict, func: Callable) -> float:
    y0 = func(x['m'], x['c'], x['s'])
    inputs = x['m'], x['c'], x['s']
    elasticities = []
    for i, input in enumerate(inputs):
        partial = jax.grad(func, argnums=i)(x['m'], x['c'], x['s'])
        elasticities.append(partial * input / y0)
    return jnp.array(elasticities)

In [17]:
elastic = elasticity(x0, ooq)

In [26]:
from jax.typing import ArrayLike

def diff_importance(elasticities: ArrayLike) -> ArrayLike:
    return jnp.round(elasticities / jnp.sum(elasticities), 3)

In [27]:
diff_importance(elastic)

Array([ 1., -1.,  1.], dtype=float32)

We find that the Differential Importance Measures of each variable have the same magnitude, with x_2 having an opposite effect as the other two inputs. 